In [ ]:
import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
    #for filename in filenames:
        #print(os.path.join(dirname, filename))

In [ ]:
os.listdir('/kaggle/input/sartorius-cell-instance-segmentation/')

# Introduction:
1. Neurological disorders, including neurodegenerative diseases such as Alzheimer's and brain tumors, are a leading cause of death and disability across the globe. However, it is hard to quantify how well these deadly disorders respond to treatment. 

2. One accepted method is to review neuronal cells via light microscopy, which is both accessible and non-invasive. Unfortunately, segmenting individual neuronal cells in microscopic images can be challenging and time-intensive. Accurate instance segmentation of these cells—with the help of computer vision—could lead to new and effective drug discoveries to treat the millions of people with these disorders.

3. Current solutions have limited accuracy for neuronal cells in particular. In internal studies to develop cell instance segmentation models, the neuroblastoma cell line SH-SY5Y consistently exhibits the lowest precision scores out of eight different cancer cell types tested. This could be because neuronal cells have a very unique, irregular and concave morphology associated with them, making them challenging to segment with commonly used mask heads.

4. In this competition, you’ll detect and delineate distinct objects of interest in biological images depicting neuronal cell types commonly used in the study of neurological disorders. More specifically, you'll use phase contrast microscopy images to train and test your model for instance segmentation of neuronal cells. Successful models will do this with a high level of accuracy.


# data
In this competition we are segmenting neuronal cells in images. The training annotations are provided as run length encoded masks, and the images are in PNG format. The number of images is small, but the number of annotated objects is quite high. The hidden test set is roughly 240 images.

Files: 
A. train.csv - IDs and masks for all training objects. None of this metadata is provided for the test set.
1. id - unique identifier for object
2. annotation - run length encoded pixels for the identified neuronal cell
3. width - source image width
4. height - source image height
5. cell_type - the cell line
6. plate_time - time plate was created

B. sample_submission.csv - a sample submission file in the correct format

C. train - train images in PNG format

D. test - test images in PNG format. Only a few test set images are available for download; the remainder can only be accessed by your notebooks when you submit.

E. train_semi_supervised - unlabeled images offered in case you want to use additional data for a semi-supervised approach.

LIVECell_dataset_2021 - A mirror of the data from the LIVECell dataset. LIVECell is the predecessor dataset to this competition. You will find extra data for the SH-SHY5Y cell line, plus several other cell lines not covered in the competition dataset that may be of interest for transfer learning.



In [ ]:
# import libraries:
import pandas as pd
import numpy as np
import glob
import cv2
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib.colors import ListedColormap

import seaborn as sns

from tqdm.notebook import tqdm


import tensorflow as tf
from tensorflow.keras.losses import BinaryCrossentropy


from statistics import mean

In [ ]:
def getImagePaths(path):
    """
    Function to Combine Directory Path with individual Image Paths
    
    parameters: path(string) - Path of directory
    returns: image_names(string) - Full Image Path
    """
    image_names = []
    for dirname, _, filenames in os.walk(path):
        for filename in tqdm(filenames):
            fullpath = os.path.join(dirname, filename)
            image_names.append(fullpath)
    return image_names

In [ ]:
#Get complete image paths for train and test datasets
DIRECTORY_PATH = "../input/sartorius-cell-instance-segmentation"
TRAIN_CSV = DIRECTORY_PATH + "/train.csv"
TRAIN_PATH = DIRECTORY_PATH + "/train"
TEST_PATH = DIRECTORY_PATH + "/test"
TRAIN_SEMI_SUPERVISED_PATH = DIRECTORY_PATH + "/train_semi_supervised"
SAMPLE_SUBMISSION_PATH = DIRECTORY_PATH + "/sample_submission.csv"

train_images_path = getImagePaths(TRAIN_PATH)
test_images_path = getImagePaths(TEST_PATH)
train_semi_supervised_path = getImagePaths(TRAIN_SEMI_SUPERVISED_PATH)

# Meta Data
The meta data is given only for the train data, meaning that they cannot be used to predict the test data but they should be used to construct a solid cross validation strategy. So let's start understanding the statistical properties of the meta data.

The meta data, which is given in the train.csv file, contains 7 categorical and 2 numerical features (see table below). Each row points to an image with the id column and its associated mask with the annotation column. The annotations are given in the "run length encoded pixels" format. Furthermore each row contains the cell type information.

In [ ]:
df_train = pd.read_csv(TRAIN_CSV)

In [ ]:
display(df_train.head())
display(df_train.shape)

In [ ]:
df_train.info()

In [ ]:
# column wise unique values:
df_train.nunique()

In [ ]:
#number of images in each directory:
print(f"Number of train images: {len(train_images_path)}")
print(f"Number of test images:  {len(test_images_path)}")

There are only 606 images in the train set, which is small for training neural network models and can easily lead to an overfitting problem. However, it's well known that this problem can be easily mitigated with the use of appropriate augmentation techniques (e.g. Ronneberger et al. 2015).

All the images have the same shape: (704 x 520) px. This is nice to have in a dataset because there won't be any complications due to a variable image resolution.

In [ ]:
print(f'Number of unique images: {df_train.id.nunique()}')
print(f'Do all the images have a width of 704: {(df_train["width"]==704).all()}')
print(f'Do all the images have a height of 520: {(df_train["height"]==520).all()}')

In [ ]:
fig, ax = plt.subplots()

instances_per_image = df_train.groupby('id').size().sort_values()

#instances_per_image.index = range(606)
#instances_per_image.median()
instances_per_image.plot.bar(ax=ax)

ax.set_xticklabels([])
ax.set_xlabel('Images')
ax.set_ylabel('Number of Instances')
plt.show()

The number of instances in each image is remarkably variable (see figure below). Some statistical measures are as follows:

1. Most of the images have more than 47 instances annotated.
2. The minimum number of instances is 4.
3. The maximum number of instances is 790.

These numbers are extremely critical to train an instance segmentation model. For instance, the famous Mask RCNN model requires the information of "maximum number of detections".

Cell Types Distribution:

Each image is annotated with one of the three cell types: shsy5y, asto, and cort. 

The distribution of the cell types is shown in the figure below. 

While the most represented cell type is shsy5y (70%) in the train set, cell types cort and astro are annotated only ~10% each. This means that the data is biased towards cell type shsy5y.

In [ ]:
#distribution of cell types in %:
cell_types = df_train.cell_type.value_counts()
cell_types = cell_types/df_train.shape[0]*100
cell_types.plot.bar()
plt.title('Cell Type Distribution')
plt.ylabel(' Percentage of instances')
plt.xlabel('Cell Type')


1. The number of unique id and cell_type combinations is equal to the number of unique images. This means that each image is associated with a unique cell type! See the result below. This also explains the distribution of the image numbers in the train.csv file. The images observed abundantly are associated with the most observed cell type shsy5y.

2. Since each image is associated with only 1 cell type, we can count the number of images associated with each cell type. The figure below shows that most of the images are associated with the cell type cort, which agrees with our previous findings.

In [ ]:
# no of imges associated with each cell type:
fig, ax = plt.subplots(1, 1)
df_train.groupby(['id','cell_type'])['cell_type'].first().value_counts().plot.bar(ax=ax)
ax.set_ylabel('Number of Images')
ax.set_xlabel('Cell Types')
fig.tight_layout()
plt.show()

In [ ]:
#distribution of plate time:
plate_time = df_train.plate_time.value_counts()
plate_time = plate_time/df_train.shape[0]*100
plate_time.plot.bar()
plt.title('Plate Time Distribution')
plt.ylabel(' Percentage of instances')
plt.xlabel('Plate Time')

In [ ]:
#distribution of elasped time:
elasped_time = df_train.elapsed_timedelta.value_counts()
elasped_time = elasped_time/df_train.shape[0]*100
elasped_time.plot.bar()
plt.title('Elasped Time Distribution')
plt.ylabel(' Percentage of instances')
plt.xlabel('Elasped Time')

# Images EDA and visualisation:
1. All images defined in train_df are of the same size - 520 * 704
2. Number of annotations per image are very varied with the minimum being 4 and maximum being 790

In [ ]:
# image size: All images defined in train_df are of the same size - 520 * 704
df_train[["height", "width"]].describe()

In [ ]:
# annotations count:
annot_counts = df_train.groupby('id')[['annotation']].count().sort_values('annotation')
annot_counts

In [ ]:
annot_counts.describe()

In [ ]:
plt.style.use('ggplot')
plt.figure(figsize = (10, 6))
plt.hist(annot_counts, bins = 50, alpha = 0.8)
plt.xlabel("Number of annotations")
plt.ylabel("Count")
plt.title("Number of annotations per image")
plt.show()

In [ ]:
def display_multiple_img(images_paths, rows, cols):
    """
    Function to Display Images from Dataset.
    
    parameters: images_path(string) - Paths of Images to be displayed
                rows(int) - No. of Rows in Output
                cols(int) - No. of Columns in Output
    """
    figure, ax = plt.subplots(nrows=rows,ncols=cols,figsize=(16,8) )
    for ind,image_path in enumerate(images_paths):
        image=cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) 
        try:
            ax.ravel()[ind].imshow(image)
            ax.ravel()[ind].set_axis_off()
        except:
            continue;
    plt.tight_layout()
    plt.show()

In [ ]:
# display train images:
display_multiple_img(train_images_path[100:150], 2,2)

In [ ]:
# display train semisupervised images:
display_multiple_img(train_semi_supervised_path[100:150], 2, 2)

In [ ]:
#display test images:
display_multiple_img(test_images_path, 1, 3)

It's time to look at the images and the masks now. The figures below shows randomly selected images corresponding to each of the three distinct cell types. Each cell type has its own unique morphological properties.

astro instances are the biggest in shape. They cover a lot of space in the masks.
cort instances are smaller than the other cell types in general and they are in circle-like shapes. They don't cover much space in the masks.
shsy5y instances are slightly bigger, elongated and more abundant than the cort instances. They cover more space than the cort cells.

In [ ]:
def make_mask(mask_files, image_shape=(520, 704), color=False):
    mask = np.zeros(image_shape).ravel()
    for i, mask_file in enumerate(mask_files):
        couples = np.array(mask_file.split()).reshape(-1, 2).astype(int)
        couples[:, 1] = couples[:, 0] + couples[:, 1]
        for couple in couples:
            if color:
                mask[couple[0]: couple[1]] = i
            else:
                mask[couple[0]: couple[1]] = 1
    mask = mask.reshape(520, 704)
    return mask

def plot_image(image_id='0030fd0e6378'):
    fig, ax = plt.subplots(1, 2, figsize=(14,5))
    cell_type = df_train.loc[df_train['id'] == image_id, 'cell_type'][0:1].values
    
    file_name = os.path.join(
        '../input/sartorius-cell-instance-segmentation',
        'train', image_id + '.png')
    image = plt.imread(file_name)
    mask_files = df_train.loc[df_train['id'] == image_id, 'annotation']
    mask = make_mask(mask_files)

    ax[0].imshow(
        image,
        cmap = plt.get_cmap('winter'), 
        origin = 'upper',
        vmax = np.quantile(image, 0.99),
        vmin = np.quantile(image, 0.05)
    )
    ax[0].set_title(f'Source [{image_id}]')
    ax[0].axis('off')
    
    ax[1].imshow(
        image,
        cmap = plt.get_cmap('winter'), 
        origin = 'upper',
        vmax = 255,
        vmin = 0)
    ax[1].imshow(mask, alpha=1, cmap=plt.get_cmap('seismic'))
    ax[1].set_title(f'Source [{image_id}] + Mask {cell_type}')
    ax[1].axis('off')
    plt.show()

select_image_ids = []
select_image_ids.append(df_train.loc[df_train['cell_type'] == 'astro', 'id'].sample(1).to_list()[0])
select_image_ids.append(df_train.loc[df_train['cell_type'] == 'cort', 'id'].sample(1).to_list()[0])
select_image_ids.append(df_train.loc[df_train['cell_type'] == 'shsy5y', 'id'].sample(1).to_list()[0])

for image_id in select_image_ids:
    plot_image(image_id)


# Data preparation:

Loading and transforming the input images and their corresponding grayscale masks¶


In [ ]:
DIRECTORY_PATH = "../input/sartorius-cell-instance-segmentation"
TRAIN_CSV = DIRECTORY_PATH + "/train.csv"
TRAIN_PATH = DIRECTORY_PATH + "/train"
TEST_PATH = DIRECTORY_PATH + "/test"
TRAIN_SEMI_SUPERVISED_PATH = DIRECTORY_PATH + "/train_semi_supervised"
SAMPLE_SUBMISSION_PATH = DIRECTORY_PATH + "/sample_submission.csv"

train_images_path = getImagePaths(TRAIN_PATH)
test_images_path = getImagePaths(TEST_PATH)
train_semi_supervised_path = getImagePaths(TRAIN_SEMI_SUPERVISED_PATH)

In [ ]:
def get_image(image_id):
    image = cv2.imread(f"../input/sartorius-cell-instance-segmentation/train/{image_id}.png", cv2.IMREAD_GRAYSCALE)
    return image.reshape(*INPUT_IMG_SHAPE, 1)


In [ ]:
# https://www.kaggle.com/c/sartorius-cell-instance-segmentation/discussion/291627
def rle_decode(mask_rle, shape=(520, 704, 1)):
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape)  # Needed to align to RLE direction

def rle_encode(img):
    pixels = img.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

In [ ]:
# each mask annotation has one area
mask = df_train[df_train["id"] == "0030fd0e6378"]["annotation"].tolist()[0]
img = rle_decode(mask)
plt.imshow(img, cmap="gray");


# Model building:


In [ ]:
sample_submission=pd.read_csv(SAMPLE_SUBMISSION_PATH)
IMG_HEIGHT = 520
IMG_WIDTH = 704
IMG_CHANNELS = 1
TRAIN_PATH = '../input/sartorius-cell-instance-segmentation/train/'

train_ids = df_train['id'].unique().tolist()
test_ids = sample_submission['id'].unique().tolist()

# Get and resize train images and masks
X_train = np.zeros((df_train['id'].nunique(), IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=np.uint8)
Y_train = np.zeros((df_train['id'].nunique(), IMG_HEIGHT, IMG_WIDTH, 1), dtype=np.bool)

In [ ]:
from tqdm import tqdm
for n, id_ in tqdm(enumerate(train_ids), total=len(train_ids)):
    path = TRAIN_PATH + id_
    img = cv2.imread(path + '.png')[:,:]
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY).astype(np.float32) -125
    img = np.expand_dims(img, axis = 2)
    X_train[n] = img
    
    labels = df_train[df_train["id"]
                        == id_]["annotation"].tolist()
    mask = np.zeros((520, 704, 1))
    for label in labels:
        mask += rle_decode(label, shape=(520, 704, 1))
    mask = mask.clip(0, 1)

    Y_train[n] = mask
print("Done")

In [ ]:
# Get and resize test images
test_images_id = []
X_test = np.zeros((sample_submission['id'].nunique(), IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=np.uint8)
for n, id_ in tqdm(enumerate(test_ids), total=len(test_ids)):
    path = TRAIN_PATH.replace('train', 'test') + id_
    img = cv2.imread(path + '.png')[:,:]
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY).astype(np.float32) -125
    img = np.expand_dims(img, axis = 2)
    X_test[n] = img
    test_images_id.append(id_)
print("Done")

In [ ]:
print(X_train.shape,Y_train.shape,X_test.shape)

In [ ]:
sample_id_num = 40
plt.imshow(X_train[sample_id_num][:,:,0], cmap = 'gray')
plt.show()
plt.imshow(Y_train[sample_id_num][:,:,0])
plt.show()

print('Input image:','Min:', X_train[sample_id_num][:,:,0].min(), '; Max:', X_train[sample_id_num][:,:,0].max(), '; Mean:', X_train[sample_id_num][:,:,0].mean())
print('Mask:','Min:', Y_train[sample_id_num][:,:,0].min(), '; Max:', Y_train[sample_id_num][:,:,0].max(), '; Mean:', Y_train[sample_id_num][:,:,0].mean())


In [ ]:
#dice_coefficient
def dice_coefficient(y_true, y_pred):
    numerator = 2 * tf.reduce_sum(y_true * y_pred)
    denominator = tf.reduce_sum(y_true + y_pred)
    return numerator / (denominator + tf.keras.backend.epsilon())

In [ ]:
# Model
model = keras.Sequential([
    # Convolutional layer 1
    keras.layers.Conv2D(filters=20, kernel_size=5, strides=1,
                  padding='same',input_shape=[IMG_WIDTH,IMG_HEIGHT,IMG_CHANNELS],
                  activation='relu'),
    keras.layers.BatchNormalization(),
    
    # Convolutional layer 2
    keras.layers.Conv2D(filters=10, kernel_size=1),

    # Convolutional layer 3
    keras.layers.Conv2D(filters=10, kernel_size=5, strides=1,
                  padding='same', activation='relu'),
    keras.layers.BatchNormalization(),

    # Convolutional layer 4
    keras.layers.Conv2D(filters=1, kernel_size=1),
])

In [ ]:
loss = BinaryCrossentropy(from_logits=True)

In [ ]:
model.compile(optimizer='adam', loss=loss)
model.summary()

In [ ]:
# Fit model
n_epochs = 10
batch_size = 32
from keras.callbacks import EarlyStopping
earlystopper = EarlyStopping(patience=20, verbose=1)

results = model.fit(X_train, Y_train, validation_split=0.15, batch_size=batch_size, epochs=n_epochs, 
                    callbacks=[earlystopper])
print("Done!")

In [ ]:
plt.figure(figsize=(14,4))
plt.plot(results.history['loss'])
plt.plot(results.history['val_loss'])
plt.title('model loss')
plt.ylabel('Loss')
plt.xlabel('epoch')
plt.legend(['loss', 'val_loss'], loc='upper right')
plt.show()

In [ ]:
print(X_train.shape)
print(Y_train.shape)

In [ ]:
preds_train = model.predict(X_train, verbose=1)

In [ ]:
preds_train.shape

In [ ]:
# Threshold predictions
preds_train_t = (preds_train > 0.5).astype(np.uint8)


In [ ]:
plt.imshow(preds_train_t[0], cmap="gray");


In [ ]:
# unoptimized and slow; any way to speed up?

def get_threshold(Y, pred):
    scores = list(pred.ravel())
    mask = list(Y.ravel())
    
    idxs=np.argsort(scores)[::-1]
    mask_sorted=np.array(mask)[idxs]
    sum_mask_one=np.cumsum(mask_sorted)
    IoU=sum_mask_one/(np.arange(1,len(mask_sorted)+1)+np.sum(mask_sorted)-sum_mask_one)
    best_IoU_idx=IoU.argmax()
    best_threshold=scores[idxs[best_IoU_idx]]
    best_IoU=IoU[best_IoU_idx]

    return best_threshold, best_IoU

In [ ]:
print(X_train.shape)
print(preds_train.shape)
print(Y_train.shape)

In [ ]:
get_threshold(Y_train[0], preds_train[0])

In [ ]:
img_thresholds = []         # one for each image
img_IoUs = []
for Y, P in tqdm(zip(Y_train, preds_train), total=Y_train.shape[0]):

    best_img_threshold, best_img_IoU = get_threshold(Y, P)
    img_thresholds.append(best_img_threshold)
    img_IoUs.append(best_img_IoU)

In [ ]:
best_threshold = np.mean(img_thresholds)
best_threshold_spread = np.std(img_thresholds)
avg_IoU = mean(img_IoUs)

print(f"Best threshold: {best_threshold:.3g} (+-{best_threshold_spread:.3g}), Avg. Train IoU: {avg_IoU:.3f}")

In [ ]:
dice_coefficient(Y_train, preds_train)

In [ ]:
pred_Y = (preds_train >= best_threshold)
    
def plot(img_Y, img_pred):
    output = np.zeros_like(img_Y)
    output = np.where((img_Y == 0) & (img_pred == 1), 1, output)
    output = np.where((img_Y == 1) & (img_pred == 0), 2, output)
    output = np.where((img_Y == 1) & (img_pred == 1), 3, output)

    plt.figure(figsize=(10,10))
    plt.imshow(output, cmap=ListedColormap(['black', 'gray', 'orange', 'green']))
    plt.xticks([])
    plt.yticks([]);


In [ ]:
N = 5
for i in range(N):
    img_Y = Y_train[i]
    img_pred = pred_Y[i]
    
    plot(img_Y, img_pred)
    plt.show()

# green: correct prediction
# gray: false positive (too much)
# orange: false negative (missed)


In [ ]:
preds_test = model.predict(X_test, verbose=1)
preds_test_t = (preds_test >= best_threshold).astype(np.uint8)

In [ ]:
preds_test_t[1].shape

In [ ]:
# Test samples
from random import randint
ix = randint(0, len(preds_test_t)-1)
print(ix)
plt.imshow(X_test[ix])
plt.show()
plt.imshow(np.squeeze(preds_test_t[ix]))
plt.show()


In [ ]:
print(preds_test_t[0].shape)
print(preds_test_t[1].shape)
print(preds_test_t[2].shape)

In [ ]:
def check_overlap(msk):
    msk = msk.astype(np.bool).astype(np.uint8)
    return np.any(np.sum(msk, axis=-1)>1)

In [ ]:
for test_mask in preds_test_t:
    print(check_overlap(test_mask))

In [ ]:
# split the mask into each cluster nucleus for the submision
# seen on https://www.kaggle.com/c/sartorius-cell-instance-segmentation/discussion/288376
def post_process(mask, min_size=80):
    num_component, component = cv2.connectedComponents(mask.astype(np.uint8))
    predictions = []
    for c in range(1, num_component):
        p = (component == c)
        if p.sum() > min_size:
            a_prediction = np.zeros((520, 704), np.float32)
            a_prediction[p] = 1
            predictions.append(a_prediction)
    return predictions

In [ ]:
# test the nucleus thing
plt.imshow(Y_train[4], cmap="gray");

In [ ]:
num_component, component = cv2.connectedComponents(Y_train[4].astype(np.uint8))
num_component

In [ ]:
plt.imshow(component, cmap="gray");

In [ ]:
compenent_1 = (component == 1)
plt.imshow(compenent_1, cmap="gray");

In [ ]:
final = post_process(Y_train[4])
final[0].shape

In [ ]:
plt.imshow(final[0], cmap="gray");

In [ ]:
# old submision
predicted2 = [rle_encode(test_mask2) for test_mask2 in preds_test_t]
len(predicted2[0])

In [ ]:
def remove_isolated_points_from_rle(strin):
    t2 = strin.split(" ")
    a = []
    for i in range(0, len(t2), 2):
        if t2[i+1]!="1":
            a.append(t2[i])
            a.append(t2[i+1])
    return ' '.join(a)

In [ ]:
predicted_filt = [remove_isolated_points_from_rle(s) for s in predicted2]

In [ ]:
# new version with the mask nucleus split
predicted_nucleus = []
test_nucleus_image_id = []

for index, s in enumerate(preds_test_t):
    nucleus = post_process(s)
    for nucl in nucleus:
        predicted_nucleus.append(nucl)
        test_nucleus_image_id.append(test_images_id[index])

In [ ]:
plt.imshow(predicted_nucleus[0], cmap="gray");

In [ ]:
predicted2 = [rle_encode(test_mask2) for test_mask2 in predicted_nucleus]
print(predicted2[0])
predicted_filt = [remove_isolated_points_from_rle(s) for s in predicted2]
print(predicted_filt[0])


In [ ]:
submit = sample_submission.copy()
#submit['predicted'] = predicted2
submit = pd.DataFrame({'id':test_nucleus_image_id, 'predicted':predicted_filt})

In [ ]:
print(submit.shape)
submit.head()

In [ ]:
submit.to_csv('submission.csv', index=False)

References:
1. https://www.kaggle.com/ishandutta/sartorius-indepth-eda-explanation-model
2. https://www.kaggle.com/tolgadincer/sartorius-eda-general-overview-and-outliers
3. https://www.kaggle.com/carlosgut/sartorius-simple-cnn-keras

